- Leia o arquivo abalone do exercicio 2
- Faça o preprocessamento do atributo categorico e do atributo de saida como no exercicio 2
- Estandardize todos os atributos numéricos. Voce pode estardartizar todo o arquivo de uma vez. 
Como discutimos em aula esse não é a coisa 100% certa, mas é um erro menor.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, binarize
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def pre_processing_data():
    # lê os dados do csv
    dataset= pd.read_csv(
        '/home/mauricio/projects/MO444/lista 3/abalone.csv').values
    # Primeiro transformo a coluna categorica (M,F,I) em numerica (0,1,2) 
    labelEncoder = LabelEncoder()
    # transforma a primeira coluna (M,F,I) em (0,1,2) 
    dataset[:,0] = labelEncoder.fit_transform(dataset[:,0])
    # Agora eu aplico o One Hot Coding na primeira coluna
    oneHotEncoder = OneHotEncoder(categorical_features=[0])
    dataset = oneHotEncoder.fit_transform(dataset).toarray()
    # Separo a ultima coluna para poder binarizar
    target = dataset[:,10].reshape(-1,1)
    # Para valores > 13 transformo em 1, para valores <=13 transformo em 0
    binarize(target,threshold=13, copy=False)
    # Para funcionar com o StratifiedKFold
    target.shape = (4176,)
    #Substituo os valores transformados no dataset original
    dataset = np.delete(dataset,-1,1)
    # Standarização
    standard_scaler = StandardScaler()
    dataset = standard_scaler.fit_transform(dataset)
    return dataset, target

X, y = pre_processing_data()
# Imprime somente os 10 primeiros dados
print(X[0:10])

[[-0.67495143 -0.68813926  1.31710822 -1.44900723 -1.43989229 -1.18425209
  -1.23034422 -1.17096695 -1.20532696 -1.21305408]
 [ 1.4815881  -0.68813926 -0.75923905  0.0498915   0.12201495 -0.10824748
  -0.30960118 -0.46361041 -0.35684354 -0.20727719]
 [-0.67495143 -0.68813926  1.31710822 -0.69955786 -0.4322102  -0.34735962
  -0.63792816 -0.64833409 -0.60773918 -0.60240383]
 [-0.67495143  1.45319423 -0.75923905 -1.61555154 -1.5406605  -1.42336423
  -1.27214983 -1.2160215  -1.28743825 -1.32081589]
 [-0.67495143  1.45319423 -0.75923905 -0.82446609 -1.08720356 -1.06469603
  -0.97339267 -0.98399054 -0.94074611 -0.85384805]
 [ 1.4815881  -0.68813926 -0.75923905  0.0498915   0.07163085  0.25042072
  -0.10465173 -0.5514668  -0.35684354  0.65481729]
 [ 1.4815881  -0.68813926 -0.75923905  0.17479973  0.17239906 -0.34735962
  -0.12402506 -0.29465583 -0.28385572  0.15192884]
 [-0.67495143 -0.68813926  1.31710822 -0.40810533 -0.38182609 -0.34735962
  -0.65118359 -0.64382864 -0.62142439 -0.53056262]


Logistic regression
- Faça o logistic regression com $C=10^{-1,0,1,2,3}$. O Loop externo deve ser um 5-fold CV estratificado. 
O loop interno para a escolha do hiperparametro deve ser um 3-fold estratificado.
- Você tem que fazer o loop interno explicitamente, usando StratifiedKFold e não funções como GridSearchCV
- Qual a acurácia do LR com a melhore escolha de parametros (para cada fold)?

In [2]:

def execute_logistic_regression(reg, X_training, X_test, 
                                     Y_training,Y_test):
    
    # Cria o classificador com C = reg e random_state = 1
    classifier = LogisticRegression(C=reg, random_state=1)
    # Treina o modelo
    classifier.fit(X_training, np.ravel(Y_training))
    # Calcula a acurácia do classificador com os dados de testes
    score = classifier.score(X_test, np.ravel(Y_test))
    return score

def internal_loop_logistic_regression(X, y):
    # Cross validation com 3-fold
    folds = 3
    # Crio o StratifiedKFold com valores default para o parâmetro 
    # suffle=False dessa forma posso rodar 100 vezes o split e ele vai 
    # gerar os mesmos 3 conjuntos
    skf = StratifiedKFold(n_splits=folds)
    skf.get_n_splits(X, y)
    # Valores de C que serão testados
    C = [1e-1, 1e0, 1e1, 1e2, 1e3]
    best_c = 0.0
    best_acc = 0.0
    for c in C:
        # Variável para acumular acurácias e calcular a media do 
        # classficador utilizando um determinado C
        acc = 0.0
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            acc += execute_logistic_regression(c, X_train, X_test, 
                                                  y_train, y_test)
        # Calcula a media
        acc = acc / folds
        # Verifica se a acurácia encontrada e a melhor que as anteriores
        if acc > best_acc:
            # Se for melhor guarda o hyperparametro C para ser usado no 
            # loop externo
            best_acc = acc
            best_c = c
    return best_c

def external_loop_logistic_regression(X, y):
    # Cross validation com 5-fold
    folds = 5
    # Crio o StratifiedKFold com valores default para o parâmetro 
    # suffle=False dessa forma posso rodar 100 vezes o split e ele vai 
    # gerar os mesmos 5 conjuntos
    skf = StratifiedKFold(n_splits=folds)
    skf.get_n_splits(X, y)
    # Variavel para acumular acurácias e calcular a media geral do 
    # classficador
    acc = 0
    # Para cada Fold
    it = 1
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Para cada conjunto de treino, submeto ao loop interno para fazer 
        # o 3-Fold e calcular o melhor hiperpârametro C
        c = internal_loop_logistic_regression(X_train, y_train)
        # Com o melhor valor de C encontrado submeto o classficador e 
        # verifico a acurácia
        cur_acc = execute_logistic_regression(c, X_train, X_test, 
                                                 y_train, y_test)
        print("Loop interno {0} - Melhor C encontrado = {1}".format(it, c))
        acc += cur_acc
        it += 1
    # Calcula a mádia
    acc_avg = acc / folds
    return acc_avg

acc = external_loop_logistic_regression(X, y)
print("Logistic Regression - Acurácia Média dos 5-fold = {0}".format(acc))

Loop interno 1 - Melhor C encontrado = 10.0
Loop interno 2 - Melhor C encontrado = 100.0
Loop interno 3 - Melhor C encontrado = 100.0
Loop interno 4 - Melhor C encontrado = 10.0
Loop interno 5 - Melhor C encontrado = 10.0
Logistic Regression - Acurácia Média dos 5-fold = 0.896314010648628


Resposta: De acordo com a documentação, se criarmos o StratifiedKFold com suffle=False 
(que significa o mesmo que deixar com o valor default) podemos rodar 100 vezes 
o split gerará conjuntos iguais em todas as execuções

Linear SVM
- Faça o LinearSVM com $C=10^{-1,0,1,2,3}$. O loop externo deve ser um 5-fold estratificado. 
O loop interno um 3-fold estratificado. Neste caso voce nao precisa fazer o 3 fold explicitamente, 
voce pode usar o GridSearchCV.
- Qual a acurácia do LinearSVM com a melhor escolha de C?


In [3]:
def internal_loop_svm(X, y):
    # Parâmetros a serem testados, nesse caso somente vamos variar o C
    param_grid = [{'C': [1e-1, 1e0, 1e1, 1e2, 1e3]}]
    svm = LinearSVC(random_state=1)
    # O Grid Search ja eh criado com valores de 3-Kfold e StratifiedKfold
    # por default
    gridSearch = GridSearchCV(svm, param_grid,cv=3)
    gridSearch.fit(X, y)
    return gridSearch.best_params_.get('C')


def external_loop_svm(X, y):
    # Cross validation com 5-fold
    folds = 5
    # Crio o StratifiedKFold com valores default para o parâmetro 
    # suffle=False dessa forma posso rodar 100 vezes o split e ele vai 
    # gerar os mesmos 5 conjuntos    
    skf = StratifiedKFold(n_splits=folds)
    skf.get_n_splits(X, y)
    # Variavel para acumular acurácias e calcular a media geral do 
    # classficador
    acc = 0
    it = 1
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Para cada conjunto de treino, submeto ao loop interno para fazer 
        # o 3-Fold e calcular o melhor hiperparâmetro C
        c = internal_loop_svm(X_train, y_train)
        # Com o melhor valor de C encontrado submeto o classficador e 
        # verifico a acurácia
        svm = LinearSVC(C=c)
        svm.fit(X_train, y_train)
        cur_acc = svm.score(X_test, y_test)
        print("Loop interno {0} - Melhor C encontrado = {1}".format(it, c))        
        acc += cur_acc
        it += 1
    # Calcula a média
    acc_avg = acc / folds
    return acc_avg

acc_svm  = external_loop_svm(X, y)
print("SVM - Acurácia Média dos 5-fold = {0}".format(acc_svm))

Loop interno 1 - Melhor C encontrado = 10.0
Loop interno 2 - Melhor C encontrado = 10.0
Loop interno 3 - Melhor C encontrado = 0.1
Loop interno 4 - Melhor C encontrado = 1.0
Loop interno 5 - Melhor C encontrado = 10.0
SVM - Acurácia Média dos 5-fold = 0.8984688418076242


Resposta: De acordo com a documentação, ao criarmos o StratifiedKFold com suffle=False  
(que significa o mesmo que deixar com o valor default) se rodarmos 100 vezes o split gerará 
conjuntos iguais em todas as execuções. No caso do SVM apesar de colocarmos o randomState=1, 
por causa da implementação do GridSearch, o resultado final
apresenta uma pequena variação a cada execução (0.8970, 0.8979, 0.8996), diferentemente dos outros

LDA
- Faça o LDA. Reporte a acurácia

In [4]:
def lda(X, y):
    # Cross validation com 5-fold
    folds = 5
    # Crio o StratifiedKFold com valores default para o parametro 
    # suffle=False dessa forma posso rodar 100 vezes o split e ele vai 
    # gerar os mesmos  5 conjuntos    
    skf = StratifiedKFold(n_splits=folds)
    skf.get_n_splits(X, y)
    acc = 0
    it = 1
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        lda = LinearDiscriminantAnalysis()
        lda.fit(X_train, y_train)
        cur_acc = lda.score(X_test, y_test)
        acc += cur_acc
        it += 1
    # Calcula a média
    acc_avg = acc / folds
    return acc_avg

acc_lda = lda(X, y)
print("LDA - Acurácia Média dos 5-fold = {0}".format(acc_lda))


LDA - Acurácia Média dos 5-fold = 0.8960713353507093


/home/mauricio/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Resposta:  De acordo com a documentação, ao criarmos o StratifiedKFold com suffle=False 
(que significa o mesmo que deixar com o valor default) se rodarmos 100 vezes o split gerará 
conjuntos iguais em todas as execuções. No caso do LDA ainda ao executarmos um warning é gerado
alertando que existem variáveis colineares, não é o objetivo do exercício,mas poderíamos 
analisar se aplicar uma redução de dimensionalidade poderíamos ter resultados ainda melhores

Classificador final
- Qual o melhor classificador para esse problema?
- Se não o LDA, calcule o hiperparametro C a ser usado
- Gere o classificador final.

Apesar de apresentar alguma variação nos resultados o SVM ainda foi um pouco melhor, apesar que todos os resultados dos classificadores ficarem muito próximos

In [5]:
c_final = internal_loop_svm(X, y)
svm = LinearSVC(C=c_final)
svm.fit(X, y)
print(svm)
print(svm.coef_)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
[[  5.55385559e-04  -1.29556095e-02   1.20276932e-02  -2.63469188e-01
    1.67873266e-01   8.69969904e-02   1.42780971e+00  -1.31588049e+00
   -2.67271610e-01   2.57234689e-01]]


Só como comparação, como faríamos o nested cross validation usando somente funções do sklearn para achar a acurácia média

In [6]:
# Parâmetros a serem testados, nesse caso somente vamos variar o C
param_grid = [{'C': [1e-1, 1e0, 1e1, 1e2, 1e3]}]
svm = LinearSVC(random_state=1)
# O Grid Search ja eh criado com valores de 3-Kfold e StratifiedKfold
# por default
gridSearch = GridSearchCV(svm, param_grid,cv=3)
gridSearch.fit(X, y)
nested_score = cross_val_score(gridSearch, X=X, y=y, cv=5)
print(nested_score.mean())


0.89822845995
